In [1]:
import open3d as o3d
import numpy as np
import os
import sys
import copy
import time
sys.path.append('..')
import open3d_tutorial as o3dtut
o3dtut.interactive  = not 'CI' in os.environ

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [26]:
def draw_result(source,target,transformation):
    source_temp =copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    # source_temp.paint_uniform_color([1,0,1])
    # target_temp.paint_uniform_color([0,1,0])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp,target_temp])



In [12]:
def pre_process_point_cloud(pcd,voxel_size=0.05):
    radius_normal = voxel_size*2
    radius_feature = voxel_size*5
    pcd_down =pcd.voxel_down_sample(voxel_size)
    pcd_down.estimate_normals(
        search_param = o3d.geometry.KDTreeSearchParamHybrid(radius = radius_normal,max_nn=30)
    )

    # Extracting the feature from 3d mesh
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature,max_nn=100)
    )
    return pcd_down,pcd_fpfh



In [10]:
def prepare_dataset(voxel_size=0.05):
    source = o3d.io.read_point_cloud('./data/pcd/cloud_bin_0.pcd')
    target =o3d.io.read_point_cloud('./data/pcd/cloud_bin_1.pcd')
    trans_init = np.identity(4)
    source.transform(trans_init)
    draw_result(source=source,target=target,transformation=np.identity(4))

    source_down,source_fpfh = pre_process_point_cloud(source)
    target_down,target_fpfh = pre_process_point_cloud(target)

    return source,target,source_down,target_down, source_fpfh,target_fpfh
    

In [14]:
source,target,source_down,target_down, source_fpfh,target_fpfh = prepare_dataset()


In [15]:
def execute_global_registration(source_down,target_down,source_fpfh,target_fpfh,voxel_size=0.05):
    distance_threshold  =voxel_size*1.5

    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down,target_down,source_fpfh,target_fpfh,True,distance_threshold,o3d.pipelines.registration.TransformationEstimationPointToPoint(False),3,[
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)

        ],
        o3d.pipelines.registration.RANSACConvergenceCriteria(100000)
    )
    return result

In [16]:
result_ransac = execute_global_registration(source_down=source_down,target_down=target_down,source_fpfh=source_fpfh,target_fpfh=target_fpfh)

In [17]:
print(result_ransac)
draw_result(source_down,target_down,result_ransac.transformation)

RegistrationResult with fitness=6.716387e-01, inlier_rmse=3.118043e-02, and correspondence_set size of 3197
Access transformation to get result.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


# Local Registration

In [22]:
def local_refinement(source,target,voxel_size=0.05):
    distance_threshold = voxel_size*0.4
    result = o3d.pipelines.registration.registration_icp(
        source,target,distance_threshold,result_ransac.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane()
    )
    return result

In [23]:
refined_result = local_refinement(source,target)

In [27]:
print(refined_result)
draw_result(source,target,refined_result.transformation)

RegistrationResult with fitness=6.210275e-01, inlier_rmse=6.565173e-03, and correspondence_set size of 123482
Access transformation to get result.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
